<a href="https://colab.research.google.com/github/kjm12misa/Google_drive/blob/main/Python/chapter_5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

last class
#### 트리의 앙상블
- LightGBM 중요! 
  + GBM -> XGBoost -> LightGBM
  + 참고 1 : 모델 개발 속도가 빨라졌나/2. 모델의 성능이 좋아졌나
- TabNet(2019)
  + 딥러닝 컨셉.

#### 랜덤 포레스트
- 결정 트리 나무 500개 심기
- 최종적인 결정은 투표 방식
  + 나무-1 : 양성, 나무-2 : 음성, 나무-3 : 양성 ... 나무-500 : 음성

#### 용어 정리 및 여러 함수 뜻 참조함.
출처 : https://inuplace.tistory.com/570

#### 데이터 불러오기

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, 
                                                                      target, 
                                                                      test_size=0.2, 
                                                                      random_state=42)

P.267
- cross_validate() 교차 검증 수행
  - 함수 안에 있는 것은 매개변수와 같은 식으로 쓰인다.
  + 데이터 세트를 교육용과 테스트용 두 부분으로 나눈다.
  + 교육 세트에 모델을 교육/테스트 세트의 모델을 검증
  + 1~3단계를 몇 번 반복(이 수치는 사용하고 있는 **CV** 방식에 따라 달라짐)
- 순서
  1. 전체 데이터 세트를 무작위로 k개의 "폴드"로 분할
  2. 데이터 집합의 각 k-fold에 대해 데이터 집합의 k – 1배에서 모델을 구축
  3. 각 예측에 표시되는 오류를 기록
  4. 각 k-폴드가 테스트 세트로 사용될 때까지 이 과정을 반복

  ? 왜 모든 식마다 n_jobs = -1이 들어가는건가?
  - 모든 수식 안마다 모든 CPU를 사용해 준다?

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [7]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- oob_score
  + oob는 out-of-bag의 약자
  + 부트스트랩 샘플링 시 선택되지 않은 샘플
  +** oob_score를 true로 하면 훈련 종료 후 oob 샘플을 기반으로 평가를 수행**

In [8]:
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


#### 그레이디언트 부스팅
- 이전 트리의 오차를 보완하는 방식
 + 깊이가 얕은 트리를 사용
 + 학습률 매개변수로 속도를 조절
 + 단점 : 속도가 느림.

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)

score = cross_validate(gb, train_input, train_target,
                       return_train_score = True, n_jobs=-1) # (True 일 때) 훈련 종료 후 return_train_score 샘플을 기반으로 평가 수행

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [12]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state=42)

score = cross_validate(gb, train_input, train_target,
                       return_train_score = True, n_jobs=-1)
# ↑score는 cross_validate 함수 안 식들으 합친 값
# 설정해 놓은 gb + train_input + train_target + True로 정한 return_train_score의 훈련 종료 후 샘플을 기반으로 모든 CPU에서 평가를 수행한다. 라는 뜻?

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


데이터 전처리/시각화 -> 기본 모형으로 전체 흐름 설계 -> 여러 모형으로 비교 대조 -> 교차 검증, 하이퍼파라미터 성능 비교,,, => 머신러닝